In [12]:
import numpy as np
import pandas as pd
from parser_random_exp import parse_results
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
data, params = parse_results()

         exp_id _try                name step     value
0  4616315d-141    1  vq_code_usage_frac    1  0.775890
1  4616315d-141    1               f1/f1    1  0.311089
2  4616315d-141    1        f1/precision    1  0.371282
3  4616315d-141    1           f1/recall    1  0.272146
4  4616315d-141    1  vq_code_usage_frac    1  0.775890


In [24]:
met = 'many_es_to_one_gt_accuracy/gan_tokens'
res = data[data['name'] == met].round(2)

In [35]:
res.groupby(['exp_id', '_try']).describe().sort_values(('value', 'max'), ascending=False)

value                                                    \
                  count   mean           std   min     25%    50%     75%   
exp_id       _try                                                           
4616315d-141 1     10.0  0.275  5.270463e-03  0.27  0.2700  0.275  0.2800   
af188d54-05d 0     10.0  0.234  5.189733e-02  0.17  0.1750  0.260  0.2800   
53ac70df-434 0     10.0  0.250  8.164966e-03  0.24  0.2425  0.250  0.2575   
             1     10.0  0.257  4.830459e-03  0.25  0.2525  0.260  0.2600   
65b6820b-b39 1     10.0  0.253  4.830459e-03  0.25  0.2500  0.250  0.2575   
af188d54-05d 1     10.0  0.225  3.135815e-02  0.16  0.2225  0.235  0.2400   
4616315d-141 0     10.0  0.233  1.059350e-02  0.22  0.2300  0.230  0.2375   
65b6820b-b39 0     10.0  0.240  5.851389e-17  0.24  0.2400  0.240  0.2400   

                         
                    max  
exp_id       _try        
4616315d-141 1     0.28  
af188d54-05d 0     0.28  
53ac70df-434 0     0.26  
             1     0.26  
65b6820b-b39 1     0.26  
af188d54-05d 1     0.26  
4616315d-141 0     0.25  
65b6820b-b39 0     0.24